In [1]:
import pandas as pd
import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel
from gensim.parsing import strip_tags, strip_numeric, \
    strip_multiple_whitespaces, stem_text, strip_punctuation, \
    remove_stopwords, preprocess_string
import re
import os

from typing import List
import spacy

2023-07-07 17:13:01.457105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
transform_to_lower = lambda s: s.lower()
remove_single_char = lambda s: re.sub(r'\s+\w{1}\s+', '', s)

cleaning_filters = [
    strip_tags,
    strip_numeric,
    strip_punctuation, 
    strip_multiple_whitespaces, 
    transform_to_lower,
    remove_stopwords,
    remove_single_char
]

In [95]:
df = pd.read_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/reduced_arxiv_papers.parquet.gzip")

In [94]:
int(df.shape[0] * 0.75) 

638707

In [73]:
df = pd.read_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/reduced_arxiv_papers.parquet.gzip").sample().reset_index(drop=True)

In [74]:
df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cleaned_abstracts,len_abstract
0,2007.00905,Song Qingheng,"Qingheng Song, Yong Zeng, Jie Xu, and Shi Jin",A Survey of Prototype and Experiment for UAV C...,"24 pages, 6 figures",None,None,None,cs.IT eess.SP math.IT,http://creativecommons.org/licenses/by-nc-sa/4.0/,Unmanned aerial vehicle (UAV) communications...,"[{'created': 'Thu, 2 Jul 2020 06:26:20 GMT', '...",2020-07-03,"[[Song, Qingheng, ], [Zeng, Yong, ], [Xu, Jie,...",unmanned aerial vehicle uav communication a...,865
1,2102.04209,Michael Stuart,Michael T. Stuart and Markus Kneer,Guilty Artificial Minds,"20 pages, 4 figures, 1 table",None,None,None,cs.CY cs.AI cs.HC,http://creativecommons.org/licenses/by/4.0/,The concepts of blameworthiness and wrongnes...,"[{'created': 'Sun, 24 Jan 2021 21:37:35 GMT', ...",2021-02-09,"[[Stuart, Michael T., ], [Kneer, Markus, ]]",concept blameworthiness wrongness fundament...,739
2,1201.5796,Denis Jerome,Denis Jerome,Organic Superconductors: when correlations and...,"41 pages, 21 figures to be published in Journa...",None,10.1007/s10948-012-1475-7,None,cond-mat.supr-con,http://arxiv.org/licenses/nonexclusive-distrib...,This survey provides a brief account for the...,"[{'created': 'Fri, 27 Jan 2012 15:24:46 GMT', ...",2012-02-21,"[[Jerome, Denis, ]]",survey provide brief account start organic ...,649
3,1511.03076,Emma Platts Miss,"George F.R. Ellis, Emma Platts, David Sloan an...",Current observations with a decaying cosmologi...,"23 pages, 11 figures",None,10.1088/1475-7516/2016/04/026,None,astro-ph.CO gr-qc hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We use the phase plane analysis technique of...,"[{'created': 'Tue, 10 Nov 2015 12:08:23 GMT', ...",2016-04-27,"[[Ellis, George F. R., ], [Platts, Emma, ], [S...",use phase plane analysis technique madsen e...,554
4,1710.02954,Kirk Bansak,Kirk Bansak,Estimating Causal Moderation Effects with Rand...,"Forthcoming, Journal of the Royal Statistical ...",None,None,None,stat.ME,http://arxiv.org/licenses/nonexclusive-distrib...,Researchers are often interested in analyzin...,"[{'created': 'Mon, 9 Oct 2017 06:34:01 GMT', '...",2020-08-25,"[[Bansak, Kirk, ]]",researcher interested analyze conditional t...,799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851605,1301.0707,Sebastian Klein,Sebastian Klein,Chow groups of tensor triangulated categories,40 pages. The presentation of the article has ...,None,None,None,math.AG math.CT math.RT,http://arxiv.org/licenses/nonexclusive-distrib...,We recall P. Balmer's definition of tensor t...,"[{'created': 'Fri, 4 Jan 2013 11:06:40 GMT', '...",2015-10-02,"[[Klein, Sebastian, ]]",recall p. balmer definition tensor triangul...,787
851606,1707.00341,Giorgos Anastasiou,"Giorgos Anastasiou, Rodrigo Olea, David Rivera...",Noether-Wald energy in Critical Gravity,"7 pages, no figures, Final version for PLB",None,10.1016/j.physletb.2018.11.021,None,hep-th gr-qc,http://arxiv.org/licenses/nonexclusive-distrib...,Criticality represents a specific point in t...,"[{'created': 'Sun, 2 Jul 2017 19:52:32 GMT', '...",2018-11-21,"[[Anastasiou, Giorgos, ], [Olea, Rodrigo, ], [...",criticality represent specific point parame...,631
851607,1610.08526,Blagoje Oblak,Blagoje Oblak,BMS Particles in Three Dimensions,"437 pages (including index), 33 figures. Appen...",None,10.1007/978-3-319-61878-4,None,hep-th gr-qc math-ph math.GR math.MP math.RT,http://arxiv.org/licenses/nonexclusive-distrib...,This thesis is devoted to the group-theoreti...,"[{'created': 'Wed, 26 Oct 2016 20:00:16 GMT', ...",2018-01-29,"[[Oblak, Blagoje, ]]",thesis devoted group theoretic aspect dimen...,542
851608,1211.6629,Philippe Joyez,Philippe Joyez,Self-consistent dynamics of a Josephson juncti...,"7 pages, 1 figure",None,10.1103/PhysRevLett.110.217003,None,cond-mat.supr-con cond-mat.mes-hall,http://arxiv.org/licenses/nonexclusive-distrib...,We derive m

In [75]:
corpus = df['cleaned_abstracts'].to_list()


In [76]:
def gensim_tokenizer(docs: List[str]):
    tokenized_docs = list()
    for doc in docs:
        processed_words = preprocess_string(doc, cleaning_filters)
        tokenized_docs.append(processed_words)
    
    return tokenized_docs

tokenized_corpus = gensim_tokenizer(corpus)

In [77]:
len(tokenized_corpus)

851610

In [55]:
def cleaning_pipe(document):
    # Invoking gensim.parsing.preprocess_string method with set of filters
    processed_words = preprocess_string(document, cleaning_filters)
    return processed_words

In [78]:
def validate_if_dictionary_exists(dictionaty_name: str) -> bool:
    dicts_folder = "models/nlp_dictionaries"
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    dict_path = f"{parent_dir}/{dicts_folder}/{dictionaty_name}"
    print(current_dir)
    print(parent_dir)
    return os.path.isfile(dict_path)
    

validate_if_dictionary_exists('30ktokens.dict')    

/Users/luis.morales/Desktop/arxiv-paper-recommender/models
/Users/luis.morales/Desktop/arxiv-paper-recommender


False

In [79]:
def get_gensim_dictionary(tokenized_docs: List[str], dict_name: str = "corpus", save_dict: bool = False):
    """
        Create dictionary of words in preprocessed corpus and saves the dict object
    """
    dictionary = corpora.Dictionary(tokenized_docs)
    if save_dict:    
        dict_lenght = len(tokenized_corpus)
        parent_folder = "/Users/luis.morales/Desktop/arxiv-paper-recommender/models/dictionaries"
        #if validate_if_dictionary_exists('30ktokens.dict'):
        dictionary.save(f'{parent_folder}/{dict_name}.dict')
        
    return dictionary

dictionary = get_gensim_dictionary(tokenized_docs=tokenized_corpus, dict_name="TextualTango", save_dict=True)

In [31]:
# def get_gensim_dictionary(tokenized_docs: List[str], dict_name: str = "corpus", save_dict: bool = False):
#     """
#         Create dictionary of words in preprocessed corpus and saves the dict object
#     """
#     dictionary = corpora.Dictionary(tokenized_docs)
#     if save_dict:    
#         dict_lenght = len(tokenized_corpus)
#         parent_folder = "/Users/luis.morales/Desktop/arxiv-paper-recommender/models/nlp_dictionaries"
#         if validate_if_dictionary_exists('30ktokens.dict'):
#             dictionary.save(f'{parent_folder}/{dict_name}.dict')
        
#     return dictionary

# dictionary = get_gensim_dictionary(tokenized_docs=tokenized_corpus, dict_name="300Ktokens", save_dict=True)

In [80]:
BoW_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokenized_corpus]

In [81]:
tfidf_model = TfidfModel(BoW_corpus)

In [82]:
tfidf_model.save("/Users/luis.morales/Desktop/arxiv-paper-recommender/models/tfidf/TextualTango.model")

## Test model

In [83]:
# index the tfidf vector of corpus as sparse matrix
from gensim import similarities
index = similarities.SparseMatrixSimilarity(tfidf_model[BoW_corpus], num_features=len(dictionary))

In [84]:
index.save("/Users/luis.morales/Desktop/arxiv-paper-recommender/models/similarities_matrix/TextualTangoSimilarities/TextualTango")

In [67]:
def get_closest_n(query, n):
    '''get the top matching docs as per cosine similarity
    between tfidf vector of query and all docs'''
    query_document = cleaning_pipe(query)
    query_bow = dictionary.doc2bow(query_document)
    sims = index[tfidf_model[query_bow]]
    top_idx = sims.argsort()[-1*n:][::-1]
    return top_idx

In [85]:
def get_recomendations_metadata(query: str, n: int, df: pd.DataFrame):
    recommendations_idxs = get_closest_n(query, n)
    return df.iloc[recommendations_idxs].reset_index(drop=True)

In [91]:
_input = "Which papers discuss the use of statistical models and Bayesian inference for uncertainty quantification and risk assessment in engineering systems?"
results_df = get_recomendations_metadata(query=_input, df=df, n=3)


for abstract in list(zip(results_df['abstract'].to_list(), results_df['title'].to_list())):
    print(f"User Request ---- : \n {_input}")
    print(f"User Request ---- : \n ")
    print(f"Title: {abstract[1]}")
    print(f"Abstract: {abstract[0]}\n")
    print(f"--------------------------")

User Request ---- : 
 Which papers discuss the use of statistical models and Bayesian inference for uncertainty quantification and risk assessment in engineering systems?
User Request ---- : 
 
Title: A framework for benchmarking uncertainty in deep regression
Abstract:   We propose a framework for the assessment of uncertainty quantification in
deep regression. The framework is based on regression problems where the
regression function is a linear combination of nonlinear functions. Basically,
any level of complexity can be realized through the choice of the nonlinear
functions and the dimensionality of their domain. Results of an uncertainty
quantification for deep regression are compared against those obtained by a
statistical reference method. The reference method utilizes knowledge of the
underlying nonlinear functions and is based on a Bayesian linear regression
using a reference prior. Reliability of uncertainty quantification is assessed
in terms of coverage probabilities, and 